## Initialization of the notebook with installation and import of correct versions of libraries.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
import numpy as np
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random
import seaborn as sns
import pandas as pd
import gc
import cv2

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [5]:
read_small = True

In [6]:
if read_small:
  !unzip drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip -d ./dataset
else:
  !unzip drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_FT_medium.zip -d ./dataset

unzip:  cannot find or open drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip, drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip.zip or drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip.ZIP.


In [7]:
datasetPath = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset"

In [8]:
%ls

drive/  sample_data/


## Setting a seed on all frameworks for reproducibility

In [9]:
# Random seed for reproducibility
seed = 1234

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Load data and preprocessing

##64x64

In [10]:
InputShape=64
def resize_set(X):
    X = X.astype(np.float32)
    resized = np.zeros((X.shape[0], InputShape, InputShape), dtype=np.float32)
    for i in range(X.shape[0]):
        resized[i] = cv2.resize(X[i], (InputShape, InputShape), interpolation=cv2.INTER_AREA)
    return resized


In [11]:
load_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/TensorflowDataset"

X_test = np.load(os.path.join(load_path, "X_test.npy"))
Y_test = np.load(os.path.join(load_path, "Y_test.npy"))

X_test_resized = resize_set(X_test)

X_test_subset = X_test_resized[:400]
Y_test_subset = Y_test[:400]

X_test_subset = np.expand_dims(X_test_subset, axis=-1)

In [12]:
print("Final shape:", X_test_subset.shape)
print("Final shape:", Y_test_subset.shape)

# Output atteso: (400, 128, 128, 1)


Final shape: (400, 64, 64, 1)
Final shape: (400, 2)


In [13]:
save_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/BoardInput/Original"
os.makedirs(save_path, exist_ok=True)

# Quantizzazione da float [0.0–1.0] → uint8 [0–255]
X_test_quant = (X_test_subset * 255).astype(np.uint8)

# Salvataggio quantizzato
np.save(os.path.join(save_path, f"X_test_size{InputShape}_400samples_255dataset.npy"), X_test_quant)
np.save(os.path.join(save_path, f"Y_test_size{InputShape}_400samples_255dataset.npy"), Y_test_subset)

print("Saved X to:", os.path.join(save_path, f"X_test_size{InputShape}_400samples_255dataset.npy"))
print("Saved Y to:", os.path.join(save_path, f"Y_test_size{InputShape}_400samples_255dataset.npy"))


Saved X to: drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/BoardInput/Original/X_test_size64_400samples_255dataset.npy
Saved Y to: drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/BoardInput/Original/Y_test_size64_400samples_255dataset.npy


In [14]:
from google.colab import files

files.download(os.path.join(save_path, f"X_test_size{InputShape}_400samples_255dataset.npy"))
files.download(os.path.join(save_path, f"Y_test_size{InputShape}_400samples_255dataset.npy"))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##128x128

In [ ]:
def resize_set(X):
    X = X.astype(np.float32)
    resized = np.zeros((X.shape[0], 128, 128), dtype=np.float32)
    for i in range(X.shape[0]):
        resized[i] = cv2.resize(X[i], (128, 128), interpolation=cv2.INTER_AREA)
    return resized


In [ ]:
load_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/TensorflowDataset"

X_test = np.load(os.path.join(load_path, "X_test.npy"))
Y_test = np.load(os.path.join(load_path, "Y_test.npy"))

X_test_resized = resize_set(X_test)

X_test_subset = X_test_resized[:400]
Y_test_subset = Y_test[:400]

X_test_subset = np.expand_dims(X_test_subset, axis=-1)

In [ ]:
print("Final shape:", X_test_subset.shape)
print("Final shape:", Y_test_subset.shape)

# Output atteso: (400, 128, 128, 1)


Final shape: (400, 128, 128, 1)
Final shape: (400, 2)


In [ ]:
save_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/BoardInput/Original"
os.makedirs(save_path, exist_ok=True)

# Quantizzazione da float [0.0–1.0] → uint8 [0–255]
X_test_quant = (X_test_subset * 255).astype(np.uint8)

# Salvataggio quantizzato
np.save(os.path.join(save_path, "X_test_size128_400samples_255dataset.npy"), X_test_quant)
np.save(os.path.join(save_path, "Y_test_size128_400samples_255dataset.npy"), Y_test_subset)

print("Saved X to:", os.path.join(save_path, "X_test_size128_400samples_255dataset.npy"))
print("Saved Y to:", os.path.join(save_path, "Y_test_size128_400samples_255dataset.npy"))


Saved X to: drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/BoardInput/Original/X_test_size128_400samples_255dataset.npy
Saved Y to: drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/BoardInput/Original/Y_test_size128_400samples_255dataset.npy


In [ ]:
from google.colab import files

files.download(os.path.join(save_path, "X_test_size128_400samples_255dataset.npy"))
files.download(os.path.join(save_path, "Y_test_size128_400samples_255dataset.npy"))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##300x300

In [ ]:
def resize_set(X):
    X = X.astype(np.float32)
    resized = np.zeros((X.shape[0], 300, 300), dtype=np.float32)
    for i in range(X.shape[0]):
        resized[i] = cv2.resize(X[i], (300, 300), interpolation=cv2.INTER_AREA)
    return resized


In [ ]:
load_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/TensorflowDataset"

X_test = np.load(os.path.join(load_path, "X_test.npy"))
Y_test = np.load(os.path.join(load_path, "Y_test.npy"))

X_test_resized = resize_set(X_test)

X_test_subset = X_test_resized[:400]
Y_test_subset = Y_test[:400]

X_test_subset = np.expand_dims(X_test_subset, axis=-1)

In [ ]:
print("Final shape:", X_test_subset.shape)
print("Final shape:", Y_test_subset.shape)

# Output atteso: (400, 128, 128, 1)


Final shape: (400, 300, 300, 1)
Final shape: (400, 2)


In [ ]:
save_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/BoardInput/Original"
os.makedirs(save_path, exist_ok=True)

# Quantizzazione da float [0.0–1.0] → uint8 [0–255]
X_test_quant = (X_test_subset * 255).astype(np.uint8)

# Salvataggio quantizzato
np.save(os.path.join(save_path, "X_test_size300_400samples_255dataset.npy"), X_test_quant)
np.save(os.path.join(save_path, "Y_test_size300_400samples_255dataset.npy"), Y_test_subset)

print("Saved X to:", os.path.join(save_path, "X_test_size300_400samples_255dataset.npy"))
print("Saved Y to:", os.path.join(save_path, "Y_test_size300_400samples_255dataset.npy"))


Saved X to: drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/BoardInput/Original/X_test_size300_400samples_255dataset.npy
Saved Y to: drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/BoardInput/Original/Y_test_size300_400samples_255dataset.npy


In [ ]:
from google.colab import files

files.download(os.path.join(save_path, "X_test_size300_400samples_255dataset.npy"))
files.download(os.path.join(save_path, "Y_test_size300_400samples_255dataset.npy"))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>